## Hyper parameter tuning by SageMaker

1. OS level setting
2. Prepare Requirements
3. Hyperparameter Tuning

**Reference**

* [Hyperparameter Tuning using SageMaker PyTorch Container](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/hyperparameter_tuning/pytorch_mnist/hpo_pytorch_mnist.ipynb)

## OS level setting

Install packages.

```
sudo yum install gcc72-c++.x86_64
sudo yum install clang
```

g++: Install & link same version of gcc.

## Prepare Requirements

In [1]:
!git pull origin master
!pip install -r ../requirements.txt

From https://github.com/icoxfog417/allennlp-sagemaker-tuning
 * branch            master     -> FETCH_HEAD
Already up-to-date.
    100% |████████████████████████████████| 5.6MB 8.0MB/s eta 0:00:01
    100% |████████████████████████████████| 317kB 36.0MB/s ta 0:00:01
    100% |████████████████████████████████| 1.4MB 20.3MB/s ta 0:00:01
    100% |████████████████████████████████| 552kB 33.4MB/s ta 0:00:01
    100% |████████████████████████████████| 17.3MB 2.5MB/s eta 0:00:01   19% |██████                          | 3.3MB 51.2MB/s eta 0:00:01
    100% |████████████████████████████████| 51kB 26.3MB/s ta 0:00:01
    100% |████████████████████████████████| 512kB 34.0MB/s ta 0:00:01
    100% |████████████████████████████████| 13.0MB 3.9MB/s eta 0:00:01
    100% |████████████████████████████████| 245kB 34.6MB/s ta 0:00:01
    100% |████████████████████████████████| 5.5MB 9.8MB/s eta 0:00:01
    100% |████████████████████████████████| 184kB 38.7MB/s ta 0:00:01
    100% |████████████████████████

  Using cached https://files.pythonhosted.org/packages/7f/5f/c48860704092933bf1c4c1574a8de1ffd16bf4fde8bab190d747598844b2/scipy-1.2.1-cp36-cp36m-manylinux1_x86_64.whl
    100% |████████████████████████████████| 5.4MB 12.1MB/s ta 0:00:01
    100% |████████████████████████████████| 245kB 40.6MB/s ta 0:00:01
    100% |████████████████████████████████| 133kB 35.4MB/s ta 0:00:01
    100% |████████████████████████████████| 1.5MB 2.4MB/s eta 0:00:01
    100% |████████████████████████████████| 2.8MB 18.2MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 27.6MB/s ta 0:00:01
    100% |████████████████████████████████| 133kB 40.8MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 28.8MB/s ta 0:00:01
    100% |████████████████████████████████| 337kB 22.8MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 28.2MB/s ta 0:00:01
    100% |████████████████████████████████| 2.3MB 19.7MB/s ta 0:00:01
    100% |████████████████████████████████| 235kB 38.6MB/s ta 0:00

    100% |████████████████████████████████| 71kB 32.8MB/s ta 0:00:01
    100% |████████████████████████████████| 952kB 16.9MB/s ta 0:00:01
    100% |████████████████████████████████| 51kB 27.4MB/s ta 0:00:01
    100% |████████████████████████████████| 645kB 32.2MB/s ta 0:00:01
    100% |████████████████████████████████| 81kB 33.0MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 24.5MB/s ta 0:00:01
    100% |████████████████████████████████| 583kB 31.6MB/s ta 0:00:01
    100% |████████████████████████████████| 92kB 34.8MB/s ta 0:00:01
    100% |████████████████████████████████| 3.2MB 14.5MB/s ta 0:00:01
    100% |████████████████████████████████| 184kB 37.1MB/s ta 0:00:01
    100% |████████████████████████████████| 2.1MB 20.7MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/77/de/47e35a97b2b05c2fadbec67d44cfcdcd09b8086951b331d82de90d2912da/jsonschema-2.6.0-py2.py3-none-any.whl
    100% |████████████████████████████████| 92kB 21.4MB/s ta 0:00:01
    10

    100% |████████████████████████████████| 122kB 23.3MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 18.2MB/s ta 0:00:01
    100% |████████████████████████████████| 163kB 23.4MB/s ta 0:00:01
    100% |████████████████████████████████| 143kB 28.8MB/s ta 0:00:01
    100% |████████████████████████████████| 9.7MB 5.2MB/s eta 0:00:01    55% |█████████████████▉              | 5.4MB 54.1MB/s eta 0:00:01
    100% |████████████████████████████████| 829kB 29.2MB/s ta 0:00:01
    100% |████████████████████████████████| 92kB 31.4MB/s ta 0:00:01
    100% |████████████████████████████████| 112kB 36.0MB/s ta 0:00:01
    100% |████████████████████████████████| 102kB 25.7MB/s a 0:00:01
    100% |████████████████████████████████| 430kB 19.0MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/f5/e8/f6bd1eee09314e7e6dee49cbe2c5e22314ccdb38db16c9fc72d2fa80d054/docker_pycreds-0.4.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/29/19/44753eab1

    19% |██████▍                         | 1.6MB 28.9MB/s eta 0:00:01

    52% |████████████████▉               | 4.3MB 60.4MB/s eta 0:00:01

    90% |█████████████████████████████   | 7.3MB 52.6MB/s eta 0:00:01

    100% |████████████████████████████████| 8.1MB 6.2MB/s eta 0:00:01
    100% |████████████████████████████████| 122kB 40.9MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/7b/7c/c9386b82a25115cccf1903441bba3cbadcfae7b678a20167347fa8ded34c/pyasn1-0.4.5-py2.py3-none-any.whl
    100% |████████████████████████████████| 163kB 40.9MB/s ta 0:00:01
  Running setup.py bdist_wheel for nltk ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/4b/c8/24/b2343664bcceb7147efeb21c0b23703a05b23fcfeaceaa2a1e
  Running setup.py bdist_wheel for numpydoc ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/ea/55/7f/3e25d754760ccd62d6796e5b2cfe25629346f52ea00753d549
  Running setup.py bdist_wheel for jsonnet ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/f0/47/51/a178b15274ed0db775a1ae9c799ce31e511609c3ab75a7dec5
  Running setup.py bdist_wheel for word2number ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/46/2f/53/5f5c1d27549

`numpy` version is butting. So execute `pip uninstall numpy` and `conda install numpy` from terminal.

In [8]:
import os
import sys
sys.path.append(os.path.join(os.path.realpath("."), "../"))

In [12]:
def execute_example():
    from example.train import train as train_fn
    
    root = 'https://raw.githubusercontent.com/allenai/allennlp/master/tutorials/tagger/'
    train_data_path = root + 'training.txt'
    validation_data_path = root + 'validation.txt'

    embedding_dim = 6
    hidden_dim = 6

    train_fn(train_data_path, validation_data_path,
                   embedding_dim, hidden_dim)

In [13]:
execute_example()

2it [00:00, 6154.52it/s]
2it [00:00, 4275.54it/s]
100%|██████████| 4/4 [00:00<00:00, 18914.56it/s]
accuracy: 0.4444, loss: 1.0713 ||: 100%|██████████| 1/1 [00:00<00:00, 76.21it/s]
accuracy: 0.4444, loss: 1.0709 ||: 100%|██████████| 1/1 [00:00<00:00, 237.02it/s]
accuracy: 0.4444, loss: 1.0706 ||: 100%|██████████| 1/1 [00:00<00:00, 134.20it/s]
accuracy: 0.4444, loss: 1.0702 ||: 100%|██████████| 1/1 [00:00<00:00, 241.33it/s]
accuracy: 0.4444, loss: 1.0699 ||: 100%|██████████| 1/1 [00:00<00:00, 133.79it/s]
accuracy: 0.4444, loss: 1.0695 ||: 100%|██████████| 1/1 [00:00<00:00, 288.68it/s]
accuracy: 0.4444, loss: 1.0692 ||: 100%|██████████| 1/1 [00:00<00:00, 135.78it/s]
accuracy: 0.4444, loss: 1.0689 ||: 100%|██████████| 1/1 [00:00<00:00, 315.98it/s]
accuracy: 0.4444, loss: 1.0686 ||: 100%|██████████| 1/1 [00:00<00:00, 133.50it/s]
accuracy: 0.4444, loss: 1.0683 ||: 100%|██████████| 1/1 [00:00<00:00, 244.91it/s]
accuracy: 0.4444, loss: 1.0679 ||: 100%|██████████| 1/1 [00:00<00:00, 128.42it/s]


accuracy: 0.4444, loss: 1.0529 ||: 100%|██████████| 1/1 [00:00<00:00, 138.08it/s]
accuracy: 0.4444, loss: 1.0531 ||: 100%|██████████| 1/1 [00:00<00:00, 282.79it/s]
accuracy: 0.4444, loss: 1.0526 ||: 100%|██████████| 1/1 [00:00<00:00, 125.63it/s]
accuracy: 0.4444, loss: 1.0529 ||: 100%|██████████| 1/1 [00:00<00:00, 236.75it/s]
accuracy: 0.4444, loss: 1.0524 ||: 100%|██████████| 1/1 [00:00<00:00, 144.22it/s]
accuracy: 0.4444, loss: 1.0526 ||: 100%|██████████| 1/1 [00:00<00:00, 375.70it/s]
accuracy: 0.4444, loss: 1.0521 ||: 100%|██████████| 1/1 [00:00<00:00, 122.06it/s]
accuracy: 0.4444, loss: 1.0524 ||: 100%|██████████| 1/1 [00:00<00:00, 252.93it/s]
accuracy: 0.4444, loss: 1.0519 ||: 100%|██████████| 1/1 [00:00<00:00, 117.06it/s]
accuracy: 0.4444, loss: 1.0521 ||: 100%|██████████| 1/1 [00:00<00:00, 256.82it/s]
accuracy: 0.4444, loss: 1.0516 ||: 100%|██████████| 1/1 [00:00<00:00, 123.47it/s]
accuracy: 0.4444, loss: 1.0519 ||: 100%|██████████| 1/1 [00:00<00:00, 238.52it/s]
accuracy: 0.4444

accuracy: 0.4444, loss: 1.0401 ||: 100%|██████████| 1/1 [00:00<00:00, 270.22it/s]
accuracy: 0.4444, loss: 1.0394 ||: 100%|██████████| 1/1 [00:00<00:00, 92.88it/s]
accuracy: 0.4444, loss: 1.0398 ||: 100%|██████████| 1/1 [00:00<00:00, 239.55it/s]
accuracy: 0.4444, loss: 1.0391 ||: 100%|██████████| 1/1 [00:00<00:00, 121.16it/s]
accuracy: 0.4444, loss: 1.0395 ||: 100%|██████████| 1/1 [00:00<00:00, 228.82it/s]
accuracy: 0.4444, loss: 1.0388 ||: 100%|██████████| 1/1 [00:00<00:00, 127.63it/s]
accuracy: 0.4444, loss: 1.0392 ||: 100%|██████████| 1/1 [00:00<00:00, 264.04it/s]
accuracy: 0.4444, loss: 1.0385 ||: 100%|██████████| 1/1 [00:00<00:00, 118.74it/s]
accuracy: 0.4444, loss: 1.0389 ||: 100%|██████████| 1/1 [00:00<00:00, 254.40it/s]
accuracy: 0.4444, loss: 1.0382 ||: 100%|██████████| 1/1 [00:00<00:00, 127.29it/s]
accuracy: 0.4444, loss: 1.0385 ||: 100%|██████████| 1/1 [00:00<00:00, 210.38it/s]
accuracy: 0.4444, loss: 1.0379 ||: 100%|██████████| 1/1 [00:00<00:00, 125.81it/s]
accuracy: 0.4444,

accuracy: 0.4444, loss: 1.0213 ||: 100%|██████████| 1/1 [00:00<00:00, 137.69it/s]
accuracy: 0.4444, loss: 1.0216 ||: 100%|██████████| 1/1 [00:00<00:00, 278.95it/s]
accuracy: 0.4444, loss: 1.0208 ||: 100%|██████████| 1/1 [00:00<00:00, 145.72it/s]
accuracy: 0.4444, loss: 1.0212 ||: 100%|██████████| 1/1 [00:00<00:00, 256.49it/s]
accuracy: 0.4444, loss: 1.0204 ||: 100%|██████████| 1/1 [00:00<00:00, 129.12it/s]
accuracy: 0.4444, loss: 1.0207 ||: 100%|██████████| 1/1 [00:00<00:00, 269.45it/s]
accuracy: 0.4444, loss: 1.0199 ||: 100%|██████████| 1/1 [00:00<00:00, 108.53it/s]
accuracy: 0.4444, loss: 1.0202 ||: 100%|██████████| 1/1 [00:00<00:00, 285.85it/s]
accuracy: 0.4444, loss: 1.0194 ||: 100%|██████████| 1/1 [00:00<00:00, 139.97it/s]
accuracy: 0.4444, loss: 1.0198 ||: 100%|██████████| 1/1 [00:00<00:00, 328.97it/s]
accuracy: 0.4444, loss: 1.0190 ||: 100%|██████████| 1/1 [00:00<00:00, 119.90it/s]
accuracy: 0.4444, loss: 1.0193 ||: 100%|██████████| 1/1 [00:00<00:00, 242.05it/s]
accuracy: 0.4444

accuracy: 0.4444, loss: 0.9915 ||: 100%|██████████| 1/1 [00:00<00:00, 323.21it/s]
accuracy: 0.4444, loss: 0.9906 ||: 100%|██████████| 1/1 [00:00<00:00, 138.13it/s]
accuracy: 0.4444, loss: 0.9907 ||: 100%|██████████| 1/1 [00:00<00:00, 329.79it/s]
accuracy: 0.4444, loss: 0.9898 ||: 100%|██████████| 1/1 [00:00<00:00, 109.13it/s]
accuracy: 0.4444, loss: 0.9898 ||: 100%|██████████| 1/1 [00:00<00:00, 255.66it/s]
accuracy: 0.4444, loss: 0.9889 ||: 100%|██████████| 1/1 [00:00<00:00, 111.22it/s]
accuracy: 0.4444, loss: 0.9890 ||: 100%|██████████| 1/1 [00:00<00:00, 263.10it/s]
accuracy: 0.4444, loss: 0.9881 ||: 100%|██████████| 1/1 [00:00<00:00, 112.07it/s]
accuracy: 0.4444, loss: 0.9881 ||: 100%|██████████| 1/1 [00:00<00:00, 257.94it/s]
accuracy: 0.4444, loss: 0.9872 ||: 100%|██████████| 1/1 [00:00<00:00, 117.62it/s]
accuracy: 0.4444, loss: 0.9873 ||: 100%|██████████| 1/1 [00:00<00:00, 297.81it/s]
accuracy: 0.4444, loss: 0.9864 ||: 100%|██████████| 1/1 [00:00<00:00, 137.75it/s]
accuracy: 0.4444

...

accuracy: 1.0000, loss: 0.0235 ||: 100%|██████████| 1/1 [00:00<00:00, 119.82it/s]
accuracy: 1.0000, loss: 0.0236 ||: 100%|██████████| 1/1 [00:00<00:00, 261.13it/s]
accuracy: 1.0000, loss: 0.0234 ||: 100%|██████████| 1/1 [00:00<00:00, 123.80it/s]
accuracy: 1.0000, loss: 0.0235 ||: 100%|██████████| 1/1 [00:00<00:00, 230.72it/s]
accuracy: 1.0000, loss: 0.0234 ||: 100%|██████████| 1/1 [00:00<00:00, 134.45it/s]
accuracy: 1.0000, loss: 0.0234 ||: 100%|██████████| 1/1 [00:00<00:00, 328.78it/s]
accuracy: 1.0000, loss: 0.0233 ||: 100%|██████████| 1/1 [00:00<00:00, 139.07it/s]
accuracy: 1.0000, loss: 0.0234 ||: 100%|██████████| 1/1 [00:00<00:00, 272.52it/s]
accuracy: 1.0000, loss: 0.0232 ||: 100%|██████████| 1/1 [00:00<00:00, 140.53it/s]
accuracy: 1.0000, loss: 0.0233 ||: 100%|██████████| 1/1 [00:00<00:00, 265.61it/s]
accuracy: 1.0000, loss: 0.0232 ||: 100%|██████████| 1/1 [00:00<00:00, 140.09it/s]
accuracy: 1.0000, loss: 0.0233 ||: 100%|██████████| 1/1 [00:00<00:00, 233.50it/s]
accuracy: 1.0000

accuracy: 1.0000, loss: 0.0210 ||: 100%|██████████| 1/1 [00:00<00:00, 275.76it/s]
accuracy: 1.0000, loss: 0.0208 ||: 100%|██████████| 1/1 [00:00<00:00, 133.06it/s]
accuracy: 1.0000, loss: 0.0209 ||: 100%|██████████| 1/1 [00:00<00:00, 258.29it/s]
accuracy: 1.0000, loss: 0.0208 ||: 100%|██████████| 1/1 [00:00<00:00, 110.70it/s]
accuracy: 1.0000, loss: 0.0209 ||: 100%|██████████| 1/1 [00:00<00:00, 223.52it/s]
accuracy: 1.0000, loss: 0.0207 ||: 100%|██████████| 1/1 [00:00<00:00, 133.52it/s]
accuracy: 1.0000, loss: 0.0208 ||: 100%|██████████| 1/1 [00:00<00:00, 240.64it/s]
accuracy: 1.0000, loss: 0.0207 ||: 100%|██████████| 1/1 [00:00<00:00, 116.24it/s]
accuracy: 1.0000, loss: 0.0208 ||: 100%|██████████| 1/1 [00:00<00:00, 249.44it/s]
accuracy: 1.0000, loss: 0.0206 ||: 100%|██████████| 1/1 [00:00<00:00, 122.91it/s]
accuracy: 1.0000, loss: 0.0207 ||: 100%|██████████| 1/1 [00:00<00:00, 238.61it/s]
accuracy: 1.0000, loss: 0.0206 ||: 100%|██████████| 1/1 [00:00<00:00, 137.74it/s]
accuracy: 1.0000

accuracy: 1.0000, loss: 0.0187 ||: 100%|██████████| 1/1 [00:00<00:00, 150.87it/s]
accuracy: 1.0000, loss: 0.0188 ||: 100%|██████████| 1/1 [00:00<00:00, 245.12it/s]
accuracy: 1.0000, loss: 0.0187 ||: 100%|██████████| 1/1 [00:00<00:00, 142.25it/s]
accuracy: 1.0000, loss: 0.0188 ||: 100%|██████████| 1/1 [00:00<00:00, 251.23it/s]
accuracy: 1.0000, loss: 0.0187 ||: 100%|██████████| 1/1 [00:00<00:00, 130.66it/s]
accuracy: 1.0000, loss: 0.0187 ||: 100%|██████████| 1/1 [00:00<00:00, 222.40it/s]
accuracy: 1.0000, loss: 0.0186 ||: 100%|██████████| 1/1 [00:00<00:00, 129.05it/s]
accuracy: 1.0000, loss: 0.0187 ||: 100%|██████████| 1/1 [00:00<00:00, 227.05it/s]
accuracy: 1.0000, loss: 0.0186 ||: 100%|██████████| 1/1 [00:00<00:00, 124.50it/s]
accuracy: 1.0000, loss: 0.0187 ||: 100%|██████████| 1/1 [00:00<00:00, 247.83it/s]
accuracy: 1.0000, loss: 0.0185 ||: 100%|██████████| 1/1 [00:00<00:00, 136.70it/s]
accuracy: 1.0000, loss: 0.0186 ||: 100%|██████████| 1/1 [00:00<00:00, 245.87it/s]
accuracy: 1.0000

accuracy: 1.0000, loss: 0.0171 ||: 100%|██████████| 1/1 [00:00<00:00, 236.70it/s]
accuracy: 1.0000, loss: 0.0170 ||: 100%|██████████| 1/1 [00:00<00:00, 145.04it/s]
accuracy: 1.0000, loss: 0.0170 ||: 100%|██████████| 1/1 [00:00<00:00, 237.26it/s]
accuracy: 1.0000, loss: 0.0169 ||: 100%|██████████| 1/1 [00:00<00:00, 147.43it/s]
accuracy: 1.0000, loss: 0.0170 ||: 100%|██████████| 1/1 [00:00<00:00, 252.73it/s]
accuracy: 1.0000, loss: 0.0169 ||: 100%|██████████| 1/1 [00:00<00:00, 133.54it/s]
accuracy: 1.0000, loss: 0.0170 ||: 100%|██████████| 1/1 [00:00<00:00, 229.01it/s]
accuracy: 1.0000, loss: 0.0169 ||: 100%|██████████| 1/1 [00:00<00:00, 114.07it/s]
accuracy: 1.0000, loss: 0.0169 ||: 100%|██████████| 1/1 [00:00<00:00, 238.29it/s]
accuracy: 1.0000, loss: 0.0168 ||: 100%|██████████| 1/1 [00:00<00:00, 121.50it/s]
accuracy: 1.0000, loss: 0.0169 ||: 100%|██████████| 1/1 [00:00<00:00, 238.50it/s]
accuracy: 1.0000, loss: 0.0168 ||: 100%|██████████| 1/1 [00:00<00:00, 135.13it/s]
accuracy: 1.0000

['DET', 'NN', 'V', 'DET', 'NN']


## Hyperparameter Tuning